In [18]:
import dialogflow_v2 as dialogflow
import os

credentials_file = 'newagent-fpxjnq-08c3138e5a20.json'  
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_file

from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(credentials_file)

In [19]:
def create_intent(project_id, display_name, training_phrases_parts, message_texts):
    """Create an intent of the given intent type."""
    #import dialogflow_v2 as dialogflow
    intents_client = dialogflow.IntentsClient(credentials=credentials)

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part = dialogflow.types.Intent.TrainingPhrase.Part(
            text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message])

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [51]:
project_id = 'newagent-fpxjnq'
language_code = ''
display_name = list(['delete', 'transfer', 'alarm'])
training_phrases_parts = list([['delete the row please.', 'delete the column please.'],
                            ['transfer 50 euro please', 'i want to make transfer', 'transfer money'],
                            ['set alarm', 'set clock at 6', 'remind me at 9pm']])
message_texts = None #'delete the value'

#credentials_file = 'NewAgent-676cf2e40a3f.json'

In [52]:
#create_intent(project_id, display_name, training_phrases_parts, message_texts)

# create intents.

In [55]:
intents_client = dialogflow.IntentsClient()
parent = intents_client.project_agent_path(project_id)

for i in range(len(display_name)):
    training_phrases = []
    for training_phrases_part in training_phrases_parts[i]:
        part = dialogflow.types.Intent.TrainingPhrase.Part(
            text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name[i],
        training_phrases=training_phrases,
        messages=[message])

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

Intent created: name: "projects/newagent-fpxjnq/agent/intents/73b71076-eb3e-4581-aefd-4ad42ccb3e92"
display_name: "delete"
priority: 500000
messages {
  text {
  }
}

Intent created: name: "projects/newagent-fpxjnq/agent/intents/4a9e1c34-4ba9-46b5-813c-f8c8bd1dce16"
display_name: "transfer"
priority: 500000
messages {
  text {
  }
}

Intent created: name: "projects/newagent-fpxjnq/agent/intents/3dba0cf7-a0e1-46e3-847b-83e401276464"
display_name: "alarm"
priority: 500000
messages {
  text {
  }
}



# training the agent model.

In [56]:
client = dialogflow.AgentsClient()

parent = client.project_path(project_id)

response = client.train_agent(parent)


# get the intent ID.

In [57]:
name = display_name[2]

intents_client = dialogflow.IntentsClient()
parent = intents_client.project_agent_path(project_id)

intents = intents_client.list_intents(parent)

intent_names = [
    intent.name for intent in intents
    if intent.display_name == name]

intent_ids = [
    intent_name.split('/')[-1] for intent_name
    in intent_names]

intent_ids

['3dba0cf7-a0e1-46e3-847b-83e401276464']

# detect the intent.

In [58]:
'''import os
import dialogflow
from google.api_core.exceptions import InvalidArgument

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_file
'''
DIALOGFLOW_PROJECT_ID = project_id
DIALOGFLOW_LANGUAGE_CODE = 'en'
SESSION_ID = '1'


#text_to_be_analyzed = "transfer money"
text_to_be_analyzed = "set alarm"


session_client = dialogflow.SessionsClient()
session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)

text_input = dialogflow.types.TextInput(text=text_to_be_analyzed, language_code=DIALOGFLOW_LANGUAGE_CODE)
query_input = dialogflow.types.QueryInput(text=text_input)

try:
    response = session_client.detect_intent(session=session, query_input=query_input)
except InvalidArgument:
    raise

print("Query text:", response.query_result.query_text)
print("Detected intent:", response.query_result.intent.display_name)
print("Detected intent confidence:", response.query_result.intent_detection_confidence)
#print("Fulfillment text:", response.query_result.fulfillment_text)


Query text: set alarm
Detected intent: alarm
Detected intent confidence: 1.0


# delete all intents.

In [60]:
display_name

['delete', 'transfer', 'alarm']

In [ ]:
intents = intents_client.list_intents(parent)
for i in intents:
    print(i.display_name)

In [61]:
client = dialogflow.IntentsClient()

parent = client.project_agent_path(project_id)

intents = intents_client.list_intents(parent)

response = client.batch_delete_intents(parent, intents)

In [62]:
intents = intents_client.list_intents(parent)
for i in intents:
    print(i.display_name)